In [ ]:
# pip install requests boto3

SyntaxError: invalid syntax (3366081047.py, line 2)

In [ ]:
import boto3

# Provide AWS credentials directly (not recommended for production)
aws_access_key_id = ''
aws_secret_access_key = ''
aws_region = ''

# Initialize CloudWatch client with your credentials
client = boto3.client(
    'events',
    region_name=aws_region,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)


In [ ]:
import requests
import boto3
import json
import base64
from urllib.parse import urlencode

# Spotify API credentials
CLIENT_ID = ''
CLIENT_SECRET = ''
REDIRECT_URI = ''  # Make sure to register this in your Spotify Developer Dashboard

# AWS Region for CloudWatch
AWS_REGION = ''  # Change to your AWS region

# CloudWatch Event source and detail-type
EVENT_SOURCE = 'custom.spotify'
EVENT_DETAIL_TYPE = 'Spotify API Trigger'

# Step 1: Generate the authorization URL
def get_authorization_url():
    # Use scope for accessing public playlists and tracks
    scope = "playlist-read-private"  # Access public playlists
    auth_url = f"https://accounts.spotify.com/authorize?client_id={CLIENT_ID}&response_type=code&redirect_uri={REDIRECT_URI}&scope={scope}"
    return auth_url

# Step 2: Exchange authorization code for access token
def get_spotify_token(auth_code):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode("utf-8"),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    payload = {
        "grant_type": "authorization_code",
        "code": auth_code,  # Authorization code from the previous step
        "redirect_uri": REDIRECT_URI
    }
    response = requests.post(url, headers=headers, data=payload)
    response_data = response.json()

    # Check for token in the response
    if 'access_token' in response_data:
        return response_data['access_token']
    else:
        print("Error:", response_data)
        return None

# Step 3: Fetch data from Spotify API (e.g., get public playlists)
def fetch_spotify_data():
    # Replace with the actual authorization code from the URL
    auth_code = ''  # Replace with the actual code after user authorization
    access_token = get_spotify_token(auth_code)

    if access_token is None:
        print("Failed to obtain access token")
        return {}

    url = "https://api.spotify.com/v1/me/playlists"  # Example endpoint for public data (replace as needed)
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    return response.json()

# Function to send custom event to CloudWatch
def send_event_to_cloudwatch(data):
    # Initialize CloudWatch client
    client = boto3.client('events', region_name=AWS_REGION)

    # Send custom event to CloudWatch
    response = client.put_events(
        Entries=[{
            'Source': EVENT_SOURCE,
            'DetailType': EVENT_DETAIL_TYPE,
            'Detail': json.dumps(data),
            'EventBusName': 'default',  # Use 'default' unless you have a custom event bus
        }]
    )
    print(f"Event sent to CloudWatch: {response}")

# Main function
def main():
    # Step 1: Get the authorization URL
    auth_url = get_authorization_url()
    print(f"Visit this URL and authorize the app: {auth_url}")
    
    # Once you get the authorization code from the URL, replace 'authorization_code_value'
    # with the actual authorization code in your code
    spotify_data = fetch_spotify_data()
    print("Spotify Data:", spotify_data)

    # Send Spotify data to CloudWatch as a custom event
    send_event_to_cloudwatch(spotify_data)

if __name__ == "__main__":
    main()


Visit this URL and authorize the app: https://accounts.spotify.com/authorize?client_id=75700d4878044ef482957253e09d3a6d&response_type=code&redirect_uri=https://open.spotify.com/&scope=playlist-read-private
Spotify Data: {'href': 'https://api.spotify.com/v1/users/31dmiry7zd6db7evtcwkvddzgdsm/playlists?offset=0&limit=50', 'limit': 50, 'next': None, 'offset': 0, 'previous': None, 'total': 3, 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/7o2BwCQUGvqkqshXh49lQL'}, 'href': 'https://api.spotify.com/v1/playlists/7o2BwCQUGvqkqshXh49lQL', 'id': '7o2BwCQUGvqkqshXh49lQL', 'images': None, 'name': 'Umair Second Playlist', 'owner': {'display_name': 'Umair Nawaz', 'external_urls': {'spotify': 'https://open.spotify.com/user/31dmiry7zd6db7evtcwkvddzgdsm'}, 'href': 'https://api.spotify.com/v1/users/31dmiry7zd6db7evtcwkvddzgdsm', 'id': '31dmiry7zd6db7evtcwkvddzgdsm', 'type': 'user', 'uri': 'spotify:user:31dmiry7zd6db7evtcwkvddzgdsm'},

In [ ]:
import requests
import base64

# Spotify API credentials
# CLIENT_ID = ""
# CLIENT_SECRET = "your_client_secret"
# REDIRECT_URI = "your_redirect_uri"

# The authorization code received after user authorization
AUTHORIZATION_CODE = ""

def get_access_token():
    """
    Exchanges the authorization code for an access token.
    """
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode("utf-8"),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    payload = {
        "grant_type": "authorization_code",
        "code": AUTHORIZATION_CODE,
        "redirect_uri": REDIRECT_URI
    }

    # Make the POST request to fetch the token
    response = requests.post(url, headers=headers, data=payload)
    if response.status_code == 200:
        data = response.json()
        print("Access Token:", data['access_token'])
        print("Refresh Token:", data.get('refresh_token', 'No refresh token provided'))
        print("Expires In:", data['expires_in'], "seconds")
        return data
    else:
        print(f"Error: {response.status_code} - {response.json()}")
        return None

# Generate the access token
token_data = get_access_token()


Access Token: BQAZNX6BfCx5lpo-q12vagIMJ96ADOe0AZbNgnsI7Ibb0H2c6SajKBR37uTCBSSxWVUGkPKiHYgW06eEr6WxUSJuIvH_Lrn-WvFplw3OmcfbUNSqDQI8P4A2n3XStQ_Hs6WDS89POLgnLDn2FeQYz2j8MDL3RnqH0KRTnbpxnearMGoRpZaSCY9UATcoZiLIADdP7TywU9ImI0CZq5LFMwMJiN4
Refresh Token: AQCINrXhm7fnigr_9jcoguJyJ2CSwmHO4a-Ykqw0kJUoL3TMWSs0vnV0U-j_tgQbxqxfavNj2h1kLH1_u-84ZmE1NjhbRsX8pf8V2Lau-JI_1Y7WqcyvSgPX4KIU4LDIphY
Expires In: 3600 seconds


In [ ]:
import requests
import base64

# Spotify API credentials
# CLIENT_ID = "your_client_id"
# CLIENT_SECRET = "your_client_secret"
# REDIRECT_URI = "your_redirect_uri"

# Authorization code (obtained after user authorization)
AUTHORIZATION_CODE = ""

# Function to exchange the authorization code for an access token
def get_access_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode("utf-8"),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    payload = {
        "grant_type": "authorization_code",
        "code": AUTHORIZATION_CODE,
        "redirect_uri": REDIRECT_URI
    }
    response = requests.post(url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        print("Error:", response.status_code, response.json())
        return None

# Function to fetch user's private playlists
def fetch_private_playlists(access_token):
    url = "https://api.spotify.com/v1/me/playlists"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.json())
        return None

# Main script
def main():
    # Step 1: Get the access token
    access_token = get_access_token()
    if not access_token:
        return

    # Step 2: Fetch private playlists
    playlists = fetch_private_playlists(access_token)
    if playlists:
        print("Playlists Data:")
        for item in playlists.get("items", []):
            print(f"Name: {item['name']}, Tracks: {item['tracks']['total']}")

if __name__ == "__main__":
    main()


Playlists Data:
Name: Umair Second Playlist, Tracks: 0
Name: Umair Playlist, Tracks: 0
Name: My Playlist #1, Tracks: 0
